# Segmenting and Clustering Neighborhoods in Toronto - 1

## Install and load the libreries

In [1]:
import pandas as pd
import numpy as np

import json # library to handle JSON files. Will be used to retreive data through the Foursquare API
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge beautifulsoup4 --yes   #install beautifulsoup, a library to scrap data from web pages
import bs4  #import the Beautifulsoup library


# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [2]:
print(bs4.__version__) #prints the Beautifulsoup library's version, to check that it has been correctly imported

4.9.1


## Scrape the data

I will use the Python method *read_html* together with the *BeautifulSoup* librery to retreive the data from the Wikipedia web page.

In [3]:
# create the list of DataFrames from all of the tables on the web page
FSA = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='bs4')

This creates a list of Data Frames, one for each table present in the web page. We inspect the result and see that what we need is just the first Data Frame.

In [4]:
FSA

[    Postal Code          District  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

In [5]:
# keep only the first DataFrame, which is the one we need
FSA = FSA[0]
FSA

,Postal Code,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Prepare the data

We hav esaved the data in the Data Frame 'FSA'. We now have to rename the columns as per in the guidelines.

In [6]:
#change the name of the columns to match the ones given in the guidelines
FSA.columns=['Postal Code', 'Borough', 'Neighborhood']
FSA

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


We now have to clean the Data Frame, dropping the rows with 'Not assigned' Borough.

In [7]:
FSA =FSA[FSA.Borough != 'Not assigned']  #drops the rows with 'Not assigned' Borough
#FSA.reset_index(inplace=True)            #reset the index
FSA

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


We now have to check if some postal codes are listed multiple times for different neighborhoods, and merge them. We can see that we have here 103 different Postal Codes, and 103 rows in the DataFrame, which means that no Postal Code is listed twice. We don't have to do anything else.

In [8]:
PostalCodes = FSA['Postal Code'].nunique()  #counts the number of different Postal Codes.
DF_rows = FSA.shape[0]

print('There are {} different Postal Codes and {} rows in the DataFrame.'.format(PostalCodes, DF_rows))

There are 103 different Postal Codes and 103 rows in the DataFrame.


We then have to check if there still are some 'Not assigned' Neighborhoods, and replace it with the name of the Borough. I will start by showing only the rows with a 'Not assigned' Neighborhood:

In [9]:
FSA[FSA['Neighborhood'] == 'Not assigned']  #checks if there are still 'Not assigned' Neighborhoods 

,Postal Code,Borough,Neighborhood


There are no 'Not assigned' Neighborhoods, so we just have to show the size of the DataFrame.

In [10]:
FSA.shape

(103, 3)